In [1]:
# !pip install stopwords

import tensorflow as tf

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [2]:
data = "imdb_reviews"

ontrue = !False

split = ["train", "test"]

(train, test) = tfds.load(data, split=split, as_supervised=ontrue)

In [3]:
import stopwords

unlist = stopwords.get_stopwords("english")

type(unlist)

list

In [4]:
train_input = []; train_label = []

test_input = []; test_label = []

retori = lambda snt : [i for i in snt.lower().split() if i not in unlist]

for data, label in train:
  retari = " ".join(retori(data.numpy().decode("utf8")))
  train_input.append(retari)
  train_label.append(label.numpy())

for data, label in test:
  retari = " ".join(retori(data.numpy().decode("utf8")))
  test_input.append(retari)
  test_label.append(label.numpy())

In [5]:
len(train_input), len(test_input), len(train_label), len(test_label)

(25000, 25000, 25000, 25000)

In [6]:
import tensorflow.keras.layers as layers

vector = layers.TextVectorization(max_tokens=10)

sample = ["aku suka ikan", "ikan goreng", "ibu masak ikan", "bapak masak ayam bersama ibu"]

vector.adapt(sample)

In [7]:
vector.get_vocabulary()[:5]

['', '[UNK]', 'ikan', 'masak', 'ibu']

In [8]:
vector.get_vocabulary()[5:]

['suka', 'goreng', 'bersama', 'bapak', 'ayam']

In [9]:
vector.get_vocabulary()[1]

'[UNK]'

In [10]:
vector.get_vocabulary(include_special_tokens=False)

['ikan', 'masak', 'ibu', 'suka', 'goreng', 'bersama', 'bapak', 'ayam']

In [11]:
samplet = ["ikan masak ibu tadi"]

vector(samplet).numpy()

array([[2, 3, 4, 1]])

In [12]:
len(vector.get_vocabulary())

10

In [13]:
max_tokens = 15000

embedding = 16

output_sequence_length = 180

tokener = layers.TextVectorization(
  max_tokens=max_tokens, output_mode="int",
  pad_to_max_tokens=ontrue,
  output_sequence_length=output_sequence_length)

tokener.adapt(train_input)

In [14]:
len(tokener.get_vocabulary())

15000

In [15]:
import numpy as np

train_input = np.array(tokener(train_input))

train_label = np.array(train_label)

In [16]:
len(train_input), len(train_label)

(25000, 25000)

In [17]:
test_input = np.array(tokener(test_input))

test_label = np.array(test_label)

In [ ]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import (
  Embedding, Dropout,
  GlobalAveragePooling1D,
  Dense
)

model = Sequential([
  Embedding(max_tokens, embedding, input_length=output_sequence_length),
  Dropout(0.2),
  GlobalAveragePooling1D(),
  Dropout(0.2),
  Dense(1, activation="sigmoid")
])

In [19]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
validation = (test_input, test_label)

plot = model.fit(train_input, train_label, epochs=10, validation_data=validation)

In [21]:
model.evaluate(train_input, train_label, verbose=0)

[0.17072126269340515, 0.9376000165939331]

In [22]:
model.evaluate(test_input, test_label, verbose=0)

[0.3093123733997345, 0.8717600107192993]

In [23]:
np.save("tokener.npy", tokener.get_vocabulary())

In [24]:
test_load = np.load("tokener.npy")

type(test_load)

numpy.ndarray

In [25]:
test_token = tf.keras.layers.TextVectorization(
  pad_to_max_tokens=ontrue,
  output_sequence_length=180,
  max_tokens = 15000
)

test_token.adapt(test_load)

In [26]:
len(test_token.get_vocabulary())

15000